In [57]:
#https://www.ratemyimmigrationjudge.com/listing/hon-john-w-cortes-immigration-judge-new-york-varick-immigration-court/

#https://www.ratemyimmigrationjudge.com/job-listings/?search_keywords=&search_location=&search_radius=50&search_lat=0&search_lng=0

In [58]:
import csv
import pandas as pd
import googlemaps
import requests 
import urllib.request
import time
from bs4 import BeautifulSoup
import pickle

In [59]:
df2 = pickle.load(open('imm_judges.p','rb'))
df2.head()

,Court,Address,Judges,Ct Admin,Street,CityZip,Phone,cz_fix,Latitude,Longitude,geometry
0,Eloy,"1705 E. Hanna Road, Suite 366\n Eloy, AZ 851...","[Cortes, John W., Gaz, Jennifer I., Habich, Pa...","Padilla, Victoria","1705 E. Hanna Road, Suite 366","Eloy, AZ 85131",520-466-3671,"[Eloy,, AZ, 85131]",32.755170,-111.553493,POINT (-111.55349 32.75517)
1,Florence,"3260 N. Pinal Parkway Avenue\n Florence, AZ ...","[Guerrero, Marni, Taylor, Bruce A.]","Toliusis, Rachel",3260 N. Pinal Parkway Avenue,"Florence, AZ 85132",520-868-3341,"[Florence,, AZ, 85132]",33.031451,-111.387343,POINT (-111.38734 33.03145)
2,Phoenix,"250 N. Seventh Ave., Suite 300\n Phoenix, AZ...","[Bartlemay Sr., Robert C., Cottor, Elizabeth A...","Riebs, Sathya","250 N. Seventh Ave., Suite 300","Phoenix, AZ 85007",602-640-2747,"[Phoenix,, AZ, 85007]",33.448437,-112.074142,POINT (-112.07414 33.44844)
3,Tucson,"300 West Congress, Suite 300\n Tucson, AZ 85...","[Davis, John W., DeAngelis, Kathryn L., Keenan...","Graff, Corey","300 West Congress, Suite 300","Tucson, AZ 85701",520-670-5212,"[Tucson,, AZ, 85701]",32.222876,-110.974848,POINT (-110.97485 32.22288)
4,Adelanto,Adelanto Detention Facility\n 10250 Rancho R...,"[DePowell, Bryan E., Halperin, Ravit R., Marcu...","Roberts, Valerie (Acting)","10250 Rancho Road, Ste. 201A","Adelanto, CA 92301",760-561-6500,"[Adelanto,, CA, 92301]",34.582770,-117.409215,POINT (-117.40922 34.58277)


# tester

In [60]:
url = 'https://www.ratemyimmigrationjudge.com/listing/hon-john-w-cortes-immigration-judge-new-york-varick-immigration-court/'
response = requests.get(url)
response

<Response [200]>

In [61]:
soup = BeautifulSoup(response.text,'html.parser')

In [62]:
bio_test = soup.findAll('p')

bio_test[0:1]

[<p>Then-Acting Attorney General Matthew Whitaker appointed John W. Cortes to begin hearingcases in March 2019. Judge Cortes earned a Bachelor of Arts in 1999 from the City University ofNew York at Brooklyn College and a Juris Doctor in 2002 at the New York Law School. From2003 to 2018, he served as an assistant district attorney for the Suffolk County DistrictAttorneyâ€™s Office, and from 2013 to 2018, he served as a principal assistant district attorney forthe same court. Judge Cortes is a member of the State Bar of Arizona and New York State Bar.</p>]

# Get Court pages

In [63]:
# https://www.ratemyimmigrationjudge.com/job-listings/?search_keywords=&search_location=eloy&search_radius=50&search_lat=0&search_lng=0

court_pages = []
for court in df2['Court']:
    url = 'https://www.ratemyimmigrationjudge.com/job-listings/?search_keywords=&search_location={}&search_radius=50&search_lat=0&search_lng=0'.format(court)
    court_pages.append(url)
court_pages[0:5]

['https://www.ratemyimmigrationjudge.com/job-listings/?search_keywords=&search_location=Eloy&search_radius=50&search_lat=0&search_lng=0',
 'https://www.ratemyimmigrationjudge.com/job-listings/?search_keywords=&search_location=Florence&search_radius=50&search_lat=0&search_lng=0',
 'https://www.ratemyimmigrationjudge.com/job-listings/?search_keywords=&search_location=Phoenix&search_radius=50&search_lat=0&search_lng=0',
 'https://www.ratemyimmigrationjudge.com/job-listings/?search_keywords=&search_location=Tucson&search_radius=50&search_lat=0&search_lng=0',
 'https://www.ratemyimmigrationjudge.com/job-listings/?search_keywords=&search_location=Adelanto&search_radius=50&search_lat=0&search_lng=0']

In [64]:

url = court_pages[0]
response = requests.get(url)
response
soup = BeautifulSoup(response.text,'html.parser')


# now judges by page

In [65]:
df2['Judges'].head()

0    [Cortes, John W., Gaz, Jennifer I., Habich, Pa...
1                  [Guerrero, Marni, Taylor, Bruce A.]
2    [Bartlemay Sr., Robert C., Cottor, Elizabeth A...
3    [Davis, John W., DeAngelis, Kathryn L., Keenan...
4    [DePowell, Bryan E., Halperin, Ravit R., Marcu...
Name: Judges, dtype: object

In [66]:
judge_list =[]

for judge in df2['Judges'][0]:
    judge_list.append(judge)
judge_list[0:20]

['Cortes, John W.',
 'Gaz, Jennifer I.',
 'Habich, Paul',
 'Penalosa Jr., Jose L.']

In [67]:
sample = df2['Judges'][0][3]

def judge_addy_part(judge):
    j_split = judge.split()
    res = [j.strip('.,') for j in j_split]
    name_num = len(res)
    # judge_hyphen =j_split
    if name_num == 1:
        judge_hyphen = res
    elif name_num == 2:
        judge_hyphen = res[1]+'-'+res[0]
    elif name_num == 3:
        judge_hyphen = res[1]+'-'+res[2]+'-'+res[0]
    else:
        judge_hyphen = res[2]+'-'+res[3]+'-'+res[0]+'-'+res[1]
    return judge_hyphen

judge_addy_part(df2['Judges'][0][3])


'Jose-L-Penalosa-Jr'

In [68]:
sample = df2['Judges'][0][3]

def judge_addy_list(list):
    j_list=[]
    for judge in list:
        j_split = judge.split()
        res = [j.strip('.,') for j in j_split]
        name_num = len(res)
        # judge_hyphen =j_split
        if name_num == 1:
            judge_hyphen = res
        elif name_num == 2:
            judge_hyphen = res[1]+'-'+res[0]
        elif name_num == 3:
            judge_hyphen = res[1]+'-'+res[2]+'-'+res[0]
        else:
            judge_hyphen = res[2]+'-'+res[3]+'-'+res[0]+'-'+res[1]
        j_list.append(judge_hyphen)
    return j_list

judge_addy_list(df2['Judges'][0])

['John-W-Cortes', 'Jennifer-I-Gaz', 'Paul-Habich', 'Jose-L-Penalosa-Jr']

In [69]:
judge_list_HY =[]
#https://www.ratemyimmigrationjudge.com/listing/hon-john-w-cortes-immigration-judge-new-york-varick-immigration-court/

for judge in df2['Judges'][0]:
    judge_list_HY.append(judge_addy_part(judge))
judge_list_HY
    
    

['John-W-Cortes', 'Jennifer-I-Gaz', 'Paul-Habich', 'Jose-L-Penalosa-Jr']

In [70]:
judge_addy_list(df2['Judges'][2])

['Robert-C-Bartlemay-Sr',
 'Elizabeth-A-Cottor',
 'LaMonte-S-Freerks',
 'Joseph-S-Imburgia',
 'Ken-Josephson',
 'Melissa-B-Karlen',
 'Munish-Sharda',
 'Linda-Spencer-Walters']

In [71]:
df2[45:52]

,Court,Address,Judges,Ct Admin,Street,CityZip,Phone,cz_fix,Latitude,Longitude,geometry
45,Saipan,Marina Heights II Building\n Suite 301\n M...,"[Clemente, Jesus, Wagner, Clarence]","Viray, Glenda (Acting)",Suite 301,Marina Heights Business Park,"Saipan, MP 96950","[Marina, Heights, Business, Park]",25.090520,55.148808,POINT (55.14881 25.09052)
46,Cleveland,801 W. Superior Avenue\n Suite 13 - 100\n ...,"[Horton, Monte J., Imbacuan, Bruce D., Owens, ...","Roder, James W.",801 W. Superior Avenue,"Cleveland, OH 44113",216-802-1100,"[Cleveland,, OH, 44113]",41.505161,-81.693445,POINT (-81.69344 41.50516)
47,Portland,"1220 SW 3rd Avenue, Suite 500\n Portland, OR...","[Hoeppner, Mindy E., Lemke, Kathy J., Zanfardi...","John, Michael","1220 SW 3rd Avenue, Suite 500","Portland, OR 97204",503-326-6341,"[Portland,, OR, 97204]",45.520247,-122.674195,POINT (-122.67419 45.52025)
48,Philadelphia,Robert Nix Federal Bldg and Courthouse\n 900...,"[Carle, John B., Ehlers, Patrick J., Garcy, An...","Alicea, Alterra","900 Market Street, Suite 504","Philadelphia, PA 19107",215-656-7000,"[Philadelphia,, PA, 19107]",39.952724,-75.163526,POINT (-75.16353 39.95272)
49,York,"Closed July 30, 2021",[],,None,None,None,None,44.933143,7.540121,POINT (7.54012 44.93314)
50,Guaynabo\n (San Juan),San Patricio Office Center\n #7 Tabonuco Str...,"[Diaz-Rex, Julia, Espinal, Pedro J., Talavera-...","Quiles, Luis","#7 Tabonuco Street, Room 401","Guaynabo, PR 00968-4605",787-749-4386,"[Guaynabo,, PR, 00968-4605]",18.423681,-66.111922,POINT (-66.11192 18.42368)
51,Memphis,"Brinkley Plaza\n 80 Monroe Ave, Suite 501\n ...","[Holt, Rebecca L., Johnson, Kelly S., Maldonad...","Davis, J. Thomas","80 Monroe Ave, Suite 501","Memphis, TN 38103",901-528-5883,"[Memphis,, TN, 38103]",35.149022,-90.051629,POINT (-90.05163 35.14902)


In [72]:
df2 = df2[df2.Court != 'York']
df2 = df2.reset_index(drop=True)
df2[45:52]

,Court,Address,Judges,Ct Admin,Street,CityZip,Phone,cz_fix,Latitude,Longitude,geometry
45,Saipan,Marina Heights II Building\n Suite 301\n M...,"[Clemente, Jesus, Wagner, Clarence]","Viray, Glenda (Acting)",Suite 301,Marina Heights Business Park,"Saipan, MP 96950","[Marina, Heights, Business, Park]",25.090520,55.148808,POINT (55.14881 25.09052)
46,Cleveland,801 W. Superior Avenue\n Suite 13 - 100\n ...,"[Horton, Monte J., Imbacuan, Bruce D., Owens, ...","Roder, James W.",801 W. Superior Avenue,"Cleveland, OH 44113",216-802-1100,"[Cleveland,, OH, 44113]",41.505161,-81.693445,POINT (-81.69344 41.50516)
47,Portland,"1220 SW 3rd Avenue, Suite 500\n Portland, OR...","[Hoeppner, Mindy E., Lemke, Kathy J., Zanfardi...","John, Michael","1220 SW 3rd Avenue, Suite 500","Portland, OR 97204",503-326-6341,"[Portland,, OR, 97204]",45.520247,-122.674195,POINT (-122.67419 45.52025)
48,Philadelphia,Robert Nix Federal Bldg and Courthouse\n 900...,"[Carle, John B., Ehlers, Patrick J., Garcy, An...","Alicea, Alterra","900 Market Street, Suite 504","Philadelphia, PA 19107",215-656-7000,"[Philadelphia,, PA, 19107]",39.952724,-75.163526,POINT (-75.16353 39.95272)
49,Guaynabo\n (San Juan),San Patricio Office Center\n #7 Tabonuco Str...,"[Diaz-Rex, Julia, Espinal, Pedro J., Talavera-...","Quiles, Luis","#7 Tabonuco Street, Room 401","Guaynabo, PR 00968-4605",787-749-4386,"[Guaynabo,, PR, 00968-4605]",18.423681,-66.111922,POINT (-66.11192 18.42368)
50,Memphis,"Brinkley Plaza\n 80 Monroe Ave, Suite 501\n ...","[Holt, Rebecca L., Johnson, Kelly S., Maldonad...","Davis, J. Thomas","80 Monroe Ave, Suite 501","Memphis, TN 38103",901-528-5883,"[Memphis,, TN, 38103]",35.149022,-90.051629,POINT (-90.05163 35.14902)
51,Conroe,"806 Hilbig Road\n Conroe, TX 77301\n 936-5...","[Brisack,Chris A., D'Andrea, Holly A., Evans, ...","Russelburg, Mark",806 Hilbig Road,"Conroe, TX 77301",936-520-5400,"[Conroe,, TX, 77301]",30.311877,-95.456051,POINT (-95.45605 30.31188)


In [73]:
# df2['Judges_Hy'] = [df2['Judges'].apply(lambda x : [judge_addy_list(x) for x in df2['Judges']])]
Judges_Hy = []
count=0
for judges in df2['Judges']:
    Judges_Hy.append(judge_addy_list(judges))
df2['Judges_Hy'] = pd.Series(Judges_Hy)
df2.head()

,Court,Address,Judges,Ct Admin,Street,CityZip,Phone,cz_fix,Latitude,Longitude,geometry,Judges_Hy
0,Eloy,"1705 E. Hanna Road, Suite 366\n Eloy, AZ 851...","[Cortes, John W., Gaz, Jennifer I., Habich, Pa...","Padilla, Victoria","1705 E. Hanna Road, Suite 366","Eloy, AZ 85131",520-466-3671,"[Eloy,, AZ, 85131]",32.755170,-111.553493,POINT (-111.55349 32.75517),"[John-W-Cortes, Jennifer-I-Gaz, Paul-Habich, J..."
1,Florence,"3260 N. Pinal Parkway Avenue\n Florence, AZ ...","[Guerrero, Marni, Taylor, Bruce A.]","Toliusis, Rachel",3260 N. Pinal Parkway Avenue,"Florence, AZ 85132",520-868-3341,"[Florence,, AZ, 85132]",33.031451,-111.387343,POINT (-111.38734 33.03145),"[Marni-Guerrero, Bruce-A-Taylor]"
2,Phoenix,"250 N. Seventh Ave., Suite 300\n Phoenix, AZ...","[Bartlemay Sr., Robert C., Cottor, Elizabeth A...","Riebs, Sathya","250 N. Seventh Ave., Suite 300","Phoenix, AZ 85007",602-640-2747,"[Phoenix,, AZ, 85007]",33.448437,-112.074142,POINT (-112.07414 33.44844),"[Robert-C-Bartlemay-Sr, Elizabeth-A-Cottor, La..."
3,Tucson,"300 West Congress, Suite 300\n Tucson, AZ 85...","[Davis, John W., DeAngelis, Kathryn L., Keenan...","Graff, Corey","300 West Congress, Suite 300","Tucson, AZ 85701",520-670-5212,"[Tucson,, AZ, 85701]",32.222876,-110.974848,POINT (-110.97485 32.22288),"[John-W-Davis, Kathryn-L-DeAngelis, Sean-H-Kee..."
4,Adelanto,Adelanto Detention Facility\n 10250 Rancho R...,"[DePowell, Bryan E., Halperin, Ravit R., Marcu...","Roberts, Valerie (Acting)","10250 Rancho Road, Ste. 201A","Adelanto, CA 92301",760-561-6500,"[Adelanto,, CA, 92301]",34.582770,-117.409215,POINT (-117.40922 34.58277),"[Bryan-E-DePowell, Ravit-R-Halperin, Steven-Ma..."


In [75]:
df2['CityZip'][0].split()[0].strip(',.')

'Eloy'

In [77]:
# good for pulling the city out of the cityzip feature.  now jsut need to hyphenate judges
judge_city =[]
for i in range(len(df2['Judges_Hy'])):
    for j in df2['Judges_Hy'][i]:
        hy_j = judge_addy_part(j)[0]
        city = df2['CityZip'][i]
        try:
            city = city.split()[0].strip(',.')
        except:
            pass
        # judge_hy = judge_addy_part(j)
        judge_city.append((hy_j,city))
judge_city[0:15]

[('John-W-Cortes', 'Eloy'),
 ('Jennifer-I-Gaz', 'Eloy'),
 ('Paul-Habich', 'Eloy'),
 ('Jose-L-Penalosa-Jr', 'Eloy'),
 ('Marni-Guerrero', 'Florence'),
 ('Bruce-A-Taylor', 'Florence'),
 ('Robert-C-Bartlemay-Sr', 'Phoenix'),
 ('Elizabeth-A-Cottor', 'Phoenix'),
 ('LaMonte-S-Freerks', 'Phoenix'),
 ('Joseph-S-Imburgia', 'Phoenix'),
 ('Ken-Josephson', 'Phoenix'),
 ('Melissa-B-Karlen', 'Phoenix'),
 ('Munish-Sharda', 'Phoenix'),
 ('Linda-Spencer-Walters', 'Phoenix'),
 ('John-W-Davis', 'Tucson')]

# now indy bios for judges

In [64]:
url3= https://www.ratemyimmigrationjudge.com/listing/hon-print-maggard-immigration-judge-san-francisco-immigration-court/
#hon-print-maggard-immigration-judge-san-francisco-immigration-court/
response3 = requests.get(url)
response3

<Response [200]>

In [69]:
soup3 = BeautifulSoup(response3.text,'html.parser')
judge_list3 = soup3.body
